# Sampling 12mer macrostates

For BICePs-directed FF parameterization, there is a step in the algorithm that requires a series of macrostate samples  to be drawn from a set of lambda-scaled energy potentials

$$ U = \sum_{{i,j} \text{in contact}} -(\epsilon_i \epsilon_j)^{1/2}$$.

for a set of force field parameters $\vec{\epsilon}$, e.g. $(3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5) $.

Below is a `MacrostateSampler()` class to achieve this sampling.

In [41]:
import numpy as np
import pandas as pd
import os, sys

class MacrostateSampler(object):
    """A class to enable macrostate sampling of the 12 mer."""
    
    def __init__(self, macrostate_infofile = './macrostates.txt',
                       macrostate_dist_dir = './macrostate_distances'):
        """Initialize the MacrostateSampler() class."""
        
        self.macrostate_infofile = macrostate_infofile
        self.macrostate_dist_dir = macrostate_dist_dir
        
        # Load in the macrostate contact state information 
        self.df = pd.read_csv(self.macrostate_infofile, sep='\t')
        self.n_macrostates = len(self.df)
        self.contact_states = [eval(s) for s in self.df['contact_state'].tolist()] # convert strings to actual lists
        self.multiplicities = np.array(self.df['multiplicity'])
        
        # Load in the macrostate_distances
        self.macrostate_distances = []
        for i in range(self.n_macrostates):
            infile = os.path.join(macrostate_dist_dir, f'macro{str(i).zfill(2)}.dist.npy')
            self.macrostate_distances.append( np.load(infile) )
            
        return

    def sample(self, nsamples, lam=1.0, epsilon=np.array([3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5])):
        """Return <nsamples> samples from the ensemble with energy function \lambda * U(x). 
        
        INPUT
            nsamples        The number of samples to draw
        
        OPTIONS
            lam             A lambda-scaling value.  Lambda is like an inverse temperature. 
                            (lam=0.0 is the uniform distribution, or infinite temperature.)   (Default: 1.0)
            epsilon         A np.array() of 12 \epsilon_i values (units kT) defining the force field parameters
            
        RETURNS
            energies        np.array() of size (N,) containing the energies of each snapshot.
                            IMPORTANT: The energies are -ln P, where P is correctly normalized.
            distances       a corresponding np.array() of size (N, 12, 12) of pairwise distances.
        """
        
        # Calculate the macrostate populations given the epsilon values
        macrostate_energies = np.zeros(self.n_macrostates)
        for i in range(self.n_macrostates):
            macrostate_energies[i] = lam*self.energy(self.contact_states[i], epsilon)
        Z_i = np.exp(-1.0*macrostate_energies)*self.multiplicities
        Z = Z_i.sum()
        macrostate_populations = Z_i/Z
        neglogP = -1.0*np.log(macrostate_populations)
        # print('macrostate_populations', macrostate_populations)
        
        # Draw N samples from the multinomial distribution
        counts = np.random.multinomial(nsamples, macrostate_populations)
        # ... these are the number of samples from each macrostate
        
        energies, distances = [], []
        for i in range(self.n_macrostates):
            for j in range(counts[i]):
                energies.append( neglogP[i] )
                distances.append( self.macrostate_distances[i] )
                        
        return np.array(energies), np.array(distances)
    

        
    def energy(self, contact_state, epsilon):
        """Given a contact state (a list of tuples for each contact) and a matrix
        of bead_pair_energies, return the energy of the chain."""
        
        bead_pair_energies_squared = np.outer(epsilon,epsilon)
        bead_pair_energies = -1.0*(bead_pair_energies_squared)**0.5
    
        result = 0.0
        for (i,j) in contact_state:
            # print('contact', i, j)
            result += bead_pair_energies[i,j]
        return result

In [56]:
s = MacrostateSampler()

if (0):
    print('s.contact_states', s.contact_states)
    print('s.multiplicities', s.multiplicities)
    print('s.n_macrostates', s.n_macrostates)
    # print('s.macrostate_distances', s.macrostate_distances)

print('Drawing 100 samples (lam=1.0, epsilon=np.array([3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5]))')
energies, distances = s.sample(100)
print('energies =', energies)
print()

print('Drawing 100 samples (lam=0.5, epsilon=np.array([3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5]))')
energies, distances = s.sample(100, lam=0.5)
print('energies =', energies)

print('Drawing 100 samples (lam=0.0, epsilon=np.array([3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5]))')
energies, distances = s.sample(100, lam=0.0)
print('energies =', energies)


print('Drawing 100 samples (lam=1.0, epsilon=np.array([1., 0, 1., 0, 1., 0, 1., 0, 0, 1., 0, 1.]))')
energies, distances = s.sample(100, lam=1.0, epsilon=np.array([1., 0, 1., 0, 1., 0, 1., 0, 0, 1., 0, 1.]))
print('energies =', energies)
print()


Drawing 100 samples (lam=1.0, epsilon=np.array([3, 0, 2, 0, 1, 0, 4, 0, 0, 1.5, 0, 5]))
len(energies) 100
energies = [4.79668681 5.18703423 4.56389902 5.91732642 5.43598489 5.21964979
 5.21964979 4.4954024  3.38904692 3.38904692 3.38904692 3.38904692
 3.38904692 4.3152566  4.3152566  4.3152566  3.40868766 3.40868766
 3.40868766 3.40868766 3.40868766 7.35706264 7.35706264 4.33281511
 4.33281511 2.79431123 2.79431123 2.79431123 2.79431123 3.59778913
 4.9043872  4.88009544 3.86877234 1.63270436 1.63270436 1.63270436
 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436
 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436
 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436 1.63270436
 1.63270436 1.86576686 1.86576686 1.86576686 1.86576686 1.86576686
 1.86576686 1.86576686 1.86576686 1.86576686 1.86576686 1.86576686
 1.86576686 1.86576686 1.86576686 1.86576686 1.86576686 1.86576686
 4.9075729  1.66018182 1.66018182 1.66018182 1.66018182 1.66018182
 1.66018182 

In [27]:
np.random.multinomial(10, [0.1, 0.1, 0.2, 0.3, 0.3])

array([2, 1, 0, 3, 4])